In [3]:
# STEP 1: Install Required Libraries
!pip install xgboost catboost

# STEP 2: Upload Files
from google.colab import files
uploaded = files.upload()  # Upload train.csv, test.csv, sample_submission.csv


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'google'

In [ ]:
# STEP 3: Import Libraries
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.multioutput import MultiOutputRegressor

# STEP 4: Load the Data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Features and Targets
X = train.iloc[:, :55]                 # First 55 columns
y = train.iloc[:, 55:]                # Last 10 columns (BlendProperty1...10)

# STEP 5: XGBoost Model
xgb_model = MultiOutputRegressor(XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    tree_method='hist'
))
xgb_model.fit(X, y)
xgb_preds = xgb_model.predict(test)

# STEP 6: CatBoost Model
catboost_model = MultiOutputRegressor(CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    verbose=0,
    random_seed=42
))
catboost_model.fit(X, y)
catboost_preds = catboost_model.predict(test)

# STEP 7: Average Ensemble
final_preds = (xgb_preds + catboost_preds) / 2

# STEP 8: Create Submission
submission = sample_submission.copy()
submission.iloc[:, 1:] = final_preds
submission.to_csv("final_submission.csv", index=False)

# STEP 9: Download the CSV
files.download("final_submission.csv")
